Librerías y conexión a la base de datos

Se importan las librerías para las bases de datos

In [1]:
import pandas as pd
import numpy as np
#import sqlite3
from openpyxl import Workbook
#from dbi import connect
import pyodbc
#from pyodbc import connect as pyodbc_connect
#from sqlite3 import connect as sqlite3_connect
from scipy import stats
from scipy import special
from scipy.stats import gmean

In [2]:
import funciones as f

In [3]:
anio = int(input('Ingrese el año: '))
mes = int(input('Ingrese el mes: '))

#anio = 2024
#mes = 3

In [4]:
#Almacenamos la información del servidor para poder realizar la conexion más adelante
server = 'ipcprod.database.windows.net'
user = 'ipcreader'
password = '1pc/*2023'
database = 'db-indices'

In [5]:
#Se crea un string para realizar la conexión, con la informacion del servidor almacenada previamente
conn_str = (
    f'Driver={{ODBC Driver 17 for SQL Server}};'
    f'Server={server};'
    f'Database={database};'
    f'Uid={user};'
    f'Pwd={password};'
)

In [6]:
try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    
    sql_command = f"EXEC [dbo].[sp_get_precios_recolectados_mes] {anio}, {mes}"
    cursor.execute(sql_command)
    boletas = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])
    
    sql_command01 = f"EXEC sp_get_indice_grupo {anio}, {mes}"
    cursor.execute(sql_command01)
    ponderaciones = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])

    conn.close()
    print("Connection successful!")
except Exception as e:
    print(f"Error: {e}")

Connection successful!


In [7]:
#boletas

___________________________________________________________________________________________

In [8]:
participacion = pd.DataFrame({
        'region': [1, 2, 3, 4, 5, 6, 7, 8],
        'participacion': [13.472203, 12.046949, 12.486967, 12.483411, 12.674423, 12.415842, 12.190962, 12.229243]
    })
participacion['region'] = participacion['region'].astype('int64')

In [9]:
bol = boletas[['region', 'codigo_articulo', 'articulo', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo']].copy()
bol.insert(1, 'cod_prod', np.where(bol['codigo_articulo'].str.len() > 8, 
                                   bol['codigo_articulo'].str.slice(0, -2), 
                                   bol['codigo_articulo'].str.slice(0, -1)))
bol.loc[:, 'cod_prod'] = bol['cod_prod'].apply(lambda x: '0' + x if len(x) == 6 else x)
bol['cantidad_anterior'] = pd.to_numeric(bol['cantidad_anterior'], errors='coerce')
bol['cantidad_actual'] = pd.to_numeric(bol['cantidad_actual'], errors='coerce')
bol['precio_anterior'] = pd.to_numeric(bol['precio_anterior'], errors='coerce')
bol['precio_actual'] = pd.to_numeric(bol['precio_actual'], errors='coerce')
bol.loc[bol['cantidad_anterior'] == 0 | bol['cantidad_anterior'].isnull(), 'cantidad_anterior'] = bol['cantidad_actual']
bol.loc[bol['precio_anterior'] == 0 | bol['precio_anterior'].isnull(), 'precio_anterior'] = bol['precio_actual']

#bol['cantidad_actual'] = bol['cantidad_anterior']
#bol['precio_actual'] = bol['precio_anterior']

bol

,region,cod_prod,codigo_articulo,articulo,cantidad_anterior,cantidad_actual,precio_anterior,precio_actual,nt_tipo
0,1,0540301,5403012,Olla y cacerola de 1 a 2lt,1.0,1.0,145.00,145.00,NaN
1,1,0312306,3123061,Vestidos formales con mangas de largo hasta la...,1.0,1.0,250.00,250.00,NaN
2,1,0721101,7211011,Llantas para automóvil 185/65 R14,1.0,1.0,400.00,375.00,NaN
3,1,0411101,4111013,Alquiler de apartamento (con o sin servicios),1.0,1.0,1700.00,1700.00,NaN
4,1,0321105,3211051,Sandalias para hombre,1.0,1.0,279.99,279.99,NaN
...,...,...,...,...,...,...,...,...,...
65126,8,1111103,11111031,Desayuno o cena completo (bebida caliente y ma...,1.0,1.0,35.00,35.00,NaN
65127,8,1111105,11111051,"Almuerzo simple (bebida, carne de res y acompa...",1.0,1.0,35.00,35.00,NaN
65128,8,1111105,11111052,"Almuerzo simple (bebida, carne de pollo y acom...",1.0,1.0,35.00,35.00,NaN
65129,8,1111105,11111053,"Almuerzo simple (bebida, carne de cerdo y acom...",1.0,1.0,35.00,35.00,NaN


In [10]:
# Se obtienen las ponderaciones de cada produto por cada region y los indices anteriores
pon_prod = f.pondcat('Producto', ponderaciones).rename(columns={'grupo_codigo':'cod_prod'})
pon_prod['cod_prod'] = pon_prod['cod_prod'].apply(lambda x: '0' + x if len(str(x)) == 6 else x)
pon_prod['ponderacion_region'] = pd.to_numeric(pon_prod['ponderacion_region'], errors='coerce')
pon_prod['indice_anterior'] = pd.to_numeric(pon_prod['indice_anterior'], errors='coerce')
#pon_prod

In [11]:
#pon_prod['ponderacion_region'].dtype
#pon_prod['ponderacion_region'].isnull().sum()

Calculo del IPC

In [12]:
# Se calcula la variacion de las variedades
variacion = f.variacion(bol)
variacion['var_prod'] = bol.apply(lambda row: 1 if row['nt_tipo'] in [351761, 351762] else row['var_prod'], axis=1)
variacion = variacion[['region', 'cod_prod', 'codigo_articulo', 'articulo', 'var_prod']]
#variacion['var_prod'].dtype
#variacion

In [13]:
variacion.to_excel('variacionpy.xlsx')

In [14]:
#relativo de precios de los productos
relativo = (variacion.groupby(['region', 'cod_prod'])
            .agg(relativo = ('var_prod', lambda x: gmean(x)))
            .reset_index())
relativo = pd.merge(relativo, pon_prod[['region', 'cod_prod', 'grupo_nombre', 'ponderacion_region', 'indice_anterior']],
         on=['region', 'cod_prod'], how='left')
relativo
#relativo['ponderacion_region'].dtype

#relativo[relativo['ponderacion_region'] == 0]

,region,cod_prod,relativo,grupo_nombre,ponderacion_region,indice_anterior
0,1,0111101,0.987911,Arroz,0.399291,105.677964
1,1,0111102,0.995504,Maíz,0.130256,99.587372
2,1,0111201,1.003824,Harina de trigo,0.072427,97.573906
3,1,0111202,1.009753,Harina de maíz,0.039212,103.633272
4,1,0111203,1.017727,Harina para atoles,0.066483,101.699437
...,...,...,...,...,...,...
3016,8,1329101,1.000000,Sombrillas y paraguas,0.050085,100.000000
3017,8,1329102,1.000000,Ataúdes,0.065264,100.000000
3018,8,1390901,1.000000,Servicios legales y financieros,0.194022,100.000000
3019,8,1390902,1.000000,Servicios funerarios,0.037005,100.000000


In [15]:
#nulos = relativo['ponderacion_region'].isnull()
#relativo[nulos]
relativo.to_excel('relativopy.xlsx')

In [16]:
#promedio ponderado
prom_pon = relativo
prom_pon['var_ac'] = prom_pon['relativo']*prom_pon['ponderacion_region']
prom_pon = prom_pon[prom_pon['var_ac'] != 0]
## lo unico que se me ocurre es hacer otros dataframes agrupados, uno para la suma de la var ac, otro para la suma de las ponderaciones y asi y luego unirlos

#prom_pon['sum_var_ac'] = prom_pon['var_ac'].sum()
prom_pon01 = prom_pon.groupby(['region', 'cod_prod', 'grupo_nombre']).agg(
    sum_var_ac=('var_ac', 'sum'),
    sum_ponderacion=('ponderacion_region', 'sum')
    ).reset_index()
prom_pon = prom_pon.merge(prom_pon01, on=['region', 'cod_prod', 'grupo_nombre'], how='left')
prom_pon['prom_pon'] = prom_pon['sum_var_ac']/prom_pon['sum_ponderacion']

#prom_pon
#prom_pon01
#prom_pon[prom_pon['var_ac'] == 0]
#prom_pon01[prom_pon01['sum_var_ac'] == 0]

In [17]:
prom_pon.to_excel('promponpy.xlsx')

Índice de precios por región

In [18]:
#índice productos
prom_pon['ind_prod'] = prom_pon['indice_anterior']*prom_pon['prom_pon']
ind_prod = f.cod_grupos(prom_pon).dropna()

ind_prod01 = ind_prod[['region', 'cod_prod', 'ind_prod']].copy()
#dist100['ind_prod'].dtype

In [19]:
ind_prod01.to_excel('indprodpy.xlsx')

In [20]:
#indice subclase
f.indices(ind_prod,'cod_subclase')

,region,grupo_codigo,ind
0,1,01111,103.106405
1,1,01112,101.494764
2,1,01113,100.368471
3,1,01114,100.362436
4,1,01115,99.740960
...,...,...,...
1195,8,13131,102.067955
1196,8,13132,100.000000
1197,8,13211,100.000000
1198,8,13291,100.000000


In [21]:
#indice clase
f.indices(ind_prod,'cod_clase')

,region,grupo_codigo,ind
0,1,0111,100.577248
1,1,0112,100.100203
2,1,0113,101.053553
3,1,0114,100.542869
4,1,0115,102.933738
...,...,...,...
662,8,1312,101.220823
663,8,1313,101.673463
664,8,1321,100.000000
665,8,1329,100.000000


In [22]:
#indice grupo
f.indices(ind_prod,'cod_grupo')

,region,grupo_codigo,ind
0,1,011,101.027560
1,1,012,100.873020
2,1,013,100.000000
3,1,021,99.523436
4,1,023,102.729898
...,...,...,...
321,8,105,101.475657
322,8,111,100.079130
323,8,131,101.279955
324,8,132,100.000000


In [23]:
#indice division
f.indices(ind_prod,'cod_div')

,region,grupo_codigo,ind
0,1,01,101.013412
1,1,02,100.403857
2,1,03,100.485813
3,1,04,99.981488
4,1,05,100.100673
...,...,...,...
95,8,08,100.464533
96,8,09,100.006735
97,8,10,100.291196
98,8,11,100.079130


Índice a nivel republica

In [24]:
#indice productos
f.indicerep(ind_prod,'cod_prod')

,cod_prod,ind_prod_rep
0,0111101,102.052889
1,0111102,98.468717
2,0111201,98.092145
3,0111202,101.150824
4,0111203,100.780782
...,...,...
431,1390901,100.000000
432,1390902,101.208090
433,1390903,100.000000
434,1390904,100.000000


In [25]:
#indice subclase
f.indicerep(ind_prod,'cod_subclase')

,cod_subclase,ind_prod_rep
0,01111,99.900455
1,01112,100.605699
2,01113,98.936064
3,01114,100.391937
4,01115,99.965233
...,...,...
163,13131,100.889009
164,13132,100.289312
165,13211,100.179359
166,13291,100.000000


In [26]:
#indice clase
f.indicerep(ind_prod,'cod_clase')

,cod_clase,ind_prod_rep
0,0111,99.244472
1,0112,100.416006
2,0113,100.684467
3,0114,97.769741
4,0115,102.307497
...,...,...
91,1312,99.744348
92,1313,100.821613
93,1321,100.179359
94,1329,100.000000


In [27]:
#indice grupo
f.indicerep(ind_prod,'cod_grupo')

,cod_grupo,ind_prod_rep
0,011,100.080154
1,012,100.969047
2,013,100.673215
3,021,99.261826
4,023,99.872162
5,031,100.290963
6,032,100.657793
7,041,100.271742
8,043,100.118016
9,044,101.442829


In [28]:
#indice division
f.indicerep(ind_prod,'cod_div')

,cod_div,ind_prod_rep
0,01,100.165892
1,02,99.385770
2,03,100.391696
3,04,99.372985
4,05,99.962979
5,06,100.215184
6,07,107.157799
7,08,100.355663
8,09,100.249066
9,10,100.487074


Indice general

In [29]:
#indice producos a nivel nacional
def indicenac(df):
    indice = df.groupby('region').apply(lambda x: pd.Series({
        'sum1': (x['ponderacion_region'] * x['ind_prod']).sum(),
        'sum2': x['ponderacion_region'].sum()
    })).reset_index()

    indice['ind_prod_rep'] = indice['sum1'] / indice['sum2']
    indice = indice[['region', 'ind_prod_rep']]

    return indice



In [30]:
indicenac(ind_prod)

,region,ind_prod_rep
0,1,101.503848
1,2,100.265602
2,3,100.857514
3,4,100.977619
4,5,100.658973
5,6,100.584372
6,7,100.345891
7,8,100.783963


In [31]:
def calculate_replicated_indicator(df):
    sum1 = (df['ponderacion_region'] * df['ind_prod']).sum()
    sum2 = df['ponderacion_region'].sum()

    result = pd.DataFrame({
        'sum1': [sum1],
        'sum2': [sum2]
    })

    result['ind_rep'] = result['sum1'] / result['sum2']

    return result

In [32]:
calculate_replicated_indicator(ind_prod)

,sum1,sum2,ind_rep
0,72593.665327,720.553965,100.747021
